In [50]:
import os
import pandas as pd
import numpy as np
import torch
import torchvision
import cv2

In [51]:
dir = '/Users/iyongjeong/WORK/dacon/clf/data/train'

In [52]:
folder_ls = os.listdir(dir)
folder_ls = sorted(folder_ls)
print(len(folder_ls))

397


In [53]:
folder_ls_fl = [folder for folder in folder_ls if not folder.startswith('.')]

In [54]:
len(folder_ls_fl)

396

In [55]:
# make file list each folder

file_ls = []
for folder in folder_ls_fl:
    folder_path = os.path.join(dir, folder)
    file_ls += [os.path.join(folder_path, file) for file in os.listdir(folder_path)]

In [56]:
print(file_ls[:5])

['/Users/iyongjeong/WORK/dacon/clf/data/train/1시리즈_F20_2013_2015/1시리즈_F20_2013_2015_0080.jpg', '/Users/iyongjeong/WORK/dacon/clf/data/train/1시리즈_F20_2013_2015/1시리즈_F20_2013_2015_0057.jpg', '/Users/iyongjeong/WORK/dacon/clf/data/train/1시리즈_F20_2013_2015/1시리즈_F20_2013_2015_0043.jpg', '/Users/iyongjeong/WORK/dacon/clf/data/train/1시리즈_F20_2013_2015/1시리즈_F20_2013_2015_0042.jpg', '/Users/iyongjeong/WORK/dacon/clf/data/train/1시리즈_F20_2013_2015/1시리즈_F20_2013_2015_0056.jpg']


In [57]:
file_ls = sorted(file_ls)

In [58]:
print(file_ls[:5])

['/Users/iyongjeong/WORK/dacon/clf/data/train/1시리즈_F20_2013_2015/1시리즈_F20_2013_2015_0000.jpg', '/Users/iyongjeong/WORK/dacon/clf/data/train/1시리즈_F20_2013_2015/1시리즈_F20_2013_2015_0001.jpg', '/Users/iyongjeong/WORK/dacon/clf/data/train/1시리즈_F20_2013_2015/1시리즈_F20_2013_2015_0002.jpg', '/Users/iyongjeong/WORK/dacon/clf/data/train/1시리즈_F20_2013_2015/1시리즈_F20_2013_2015_0003.jpg', '/Users/iyongjeong/WORK/dacon/clf/data/train/1시리즈_F20_2013_2015/1시리즈_F20_2013_2015_0004.jpg']


In [59]:
# make dataframe

df = pd.DataFrame(file_ls, columns=['image_path'])
df['image_name'] = df['image_path'].apply(lambda x: os.path.basename(x))
df['folder_name'] = df['image_path'].apply(lambda x: os.path.basename(os.path.dirname(x)))

In [60]:
# make label encoding by folder name

df['label'] = df['folder_name'].astype('category').cat.codes

In [49]:
df.to_csv('/Users/iyongjeong/WORK/dacon/clf/data/train_csv/train_file_list.csv', index=False)

In [62]:
df[:100].to_csv('/Users/iyongjeong/WORK/dacon/clf/data/train_csv/train_file_list_100.csv', index=False)